<a href="https://colab.research.google.com/github/lopezbec/AI_Gamification_Python/blob/main/reviews/appreviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
#@title Installing needed dependencies!
from IPython.display import clear_output
try:
    from app_store_scraper import AppStore
    from pprint import pprint
    import pandas as pd
    import numpy as np
    import json
    import pycountry
    import langid
    import requests
    from bs4 import BeautifulSoup
    import re
    from urllib.parse import urlparse, parse_qs
    print("Modules required are already installed!")
except ModuleNotFoundError:
    %pip install pycountry
    clear_output()
    %pip install google-play-scraper
    clear_output()
    %pip install app-store-scraper
    clear_output()
    %pip install google-search-scraper-python
    clear_output()
    %pip install pandas
    clear_output()
    %pip install langid
    clear_output()
    %pip install beautifulsoup4
    clear_output()
    %pip install pycopy-urllib.parse
    clear_output()
    print("All modules required were sucessfully installed!")

All modules required were sucessfully installed!


In [ ]:
#@title App Reviews from App Store
from IPython.display import clear_output
try:
    from app_store_scraper import AppStore
    from pprint import pprint
    import pandas as pd
    import numpy as np
    import json
    print("Modules required are already installed!")
except ModuleNotFoundError:
    %pip install app-store-scraper
    clear_output()
    %pip install google-search-scraper-python
    clear_output()
    %pip install pandas
    clear_output()
    print("All modules required were sucessfully installed!")

#@markdown * ### Input the name of the app: 
app = "tiktok" #@param {type:"string"}
applereviews = AppStore(country="us", app_name=app)
#@markdown * ### How many reviews?
count = 1001 #@param {type:"number"}
applereviews.review(how_many=count)
applereviews.reviews
applestoredf = pd.DataFrame(np.array(applereviews.reviews),columns=['review'])
applestoredf2 = applestoredf.join(pd.DataFrame(applestoredf.pop('review').tolist()))
#@markdown * ### Display data?
visualize = "Yes" #@param ["Yes", "No"]
if visualize == "Yes":
  display(applestoredf2)
else:
  print("No display")
#@markdown * ### Save to csv?
csv = "Yes" #@param ["Yes", "No"]
if csv == "Yes":
  applestoredf2.to_csv('applestore-review.csv') 
else:
  print("No export")

In [53]:
#@title App Reviews from Google Store
from IPython.display import clear_output
try:
    import langid
    from google_play_scraper import app, search, Sort, reviews, reviews_all
    from pprint import pprint
    import pandas as pd
    import pycountry
    import numpy as np
    import json
    import pdb
    import requests
    from bs4 import BeautifulSoup
    import re
    from urllib.parse import urlparse, parse_qs
except ModuleNotFoundError:
    %pip install google-play-scraper
    clear_output()
    %pip install google-search-scraper-python
    clear_output()
    %pip install pandas
    clear_output()
    %pip install pycountry
    clear_output()
    
#static variables:
langid.set_languages(['en','es'])
language = 'en'
countries = 'us'
appid = []
idiomas = ['en', 'es']
arra = ['AR', 'CL', 'CO', 'CR', 'CU', 'DO', 'EC', 'SV', 'GT', 'HN', 'MX', 'NI', 'PA', 'PY', 'PE', 'UY', 'PR', 'ES', 'GQ', 'US']
df3 = pd.DataFrame()

def searching (queryy):
  query = queryy
  url = f"https://www.google.com/search?q={query}&num=10"
  headers = {
      'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
  }
  links = []
  response = requests.get(url, headers=headers)
  html_content = response.content
  soup = BeautifulSoup(html_content, 'html.parser')
  search_results = soup.find_all('div', {'class': 'g'})
  for result in search_results:
      link = result.find('a')['href']
      links.append(link)
  pattern = re.compile(r'^https://play.google.com')
  filtered_urls = [url for url in links if pattern.match(url)]
  jbl = filtered_urls[0]
  parsed_url = urlparse(jbl)
  query_params = parse_qs(parsed_url.query)
  app_id = query_params.get('id')[0]
  return app_id

def detect_language(text):
    try:
        return langid.classify(text)[0]
    except:
        return None

def searchforCountry(searchCountry):
  countryName = pycountry.countries.get(name=searchCountry)
  return countryName.alpha_2

def searches(nameofSearch):
  result = search(
    nameofSearch,
    lang="en",  # defaults to 'en'
    country="us",  # defaults to 'us'
    n_hits=3  # defaults to 30 (= Google's maximum)
)
  return result

def getAppId(r):
  for d in r:
    id = (d['appId'])
    # appid.append(id)
    return id

def insertId(g, cont, rating, choice, leng):
  if choice == None:
    if rating == None:
      result, continuation_token = reviews(
      g,
      lang=leng, # defaults to 'en'
      country='us', # defaults to 'us'
      sort=Sort.NEWEST, # defaults to Sort.NEWEST
      count=cont # defaults to 100
      # defaults to None(means all score)
    )
      return result
    else:
      result, continuation_token = reviews(
      g,
      lang=leng, # defaults to 'en'
      country='us', # defaults to 'us'
      sort=Sort.NEWEST, # defaults to Sort.NEWEST
      count=cont, # defaults to 100
      filter_score_with=rating# defaults to None(means all score)
    )
      return result
 
  else:
    if rating == None:
      result = reviews_all(
      g,
      sleep_milliseconds=0, # defaults to 0
      lang=leng, # defaults to 'en'
      country='us', # defaults to 'us'
      sort=Sort.NEWEST, # defaults to Sort.MOST_RELEVANT
      filter_score_with=None
      )
      return result
    else: 
      result = reviews_all(
      g,
      sleep_milliseconds=0, # defaults to 0
      lang=leng, # defaults to 'en'
      country='us', # defaults to 'us'
      sort=Sort.NEWEST, # defaults to Sort.MOST_RELEVANT
      filter_score_with=rating # defaults to None(means all score)
      )
      return result

def googleplay(a, ct, r, c, l):
  searches1 = searching(a)
  # appId = getAppId(searches1)
  reviews = insertId(searches1, ct, r, c, l,)
  return reviews

#@markdown * ### Input the name of the app: 
app = "sololearn" #@param {type:"string"}

#@markdown * ### All reviews?
choice = "No" #@param ["Yes", "No"]
if choice == "Yes":
  choice = "Yes"
  count = None
else:  
  choice = None
  #@markdown * ### How many reviews? (leave empty if you chose Yes to all reviews)
  count = 100 #@param {type:"number"}
#@markdown * ### Include specific rating?
cond = "No" #@param ["Yes", "No"]  
if cond == "Yes":
  #@markdown * ### Input the rating: (0-5) (leave empty if you chose No to specific rating)
  rating = 0 #@param {type:"integer"}
if cond == "No":
  rating = None

# #@markdown * ### All countries?
# choiceCountry = "No" #@param ["Yes", "No"]

# #@markdown * ### Input the name of the Country (leave empty if you chose Yes to all countries): 
# searchCountry = "sololearn" #@param {type:"string"}

#@markdown * ### Filter by Language:
languages = "Both" #@param ["English", "Spanish", "Both"]
if languages == "English":
  idiom = 'en'
  df1 = pd.DataFrame(np.array(googleplay(app, count, rating, choice, idiom)),columns=['review'])
  df = df1.join(pd.DataFrame(df1.pop('review').tolist()))
  # Apply the detect_language function to each review
  df['language'] = df['content'].apply(detect_language)

  # # # Filter the dataframe for English reviews only
  df3 = df[df['language'] == 'en']

if languages == "Spanish":
  idioma = 'es'
  df1 = pd.DataFrame(np.array(googleplay(app, count, rating, choice, idioma)),columns=['review'])
  df = df1.join(pd.DataFrame(df1.pop('review').tolist()))
  df['language'] = df['content'].apply(detect_language)

  # Filter the dataframe for English reviews only
  df3 = df[df['language'] == 'es']

#@markdown * ### Order by language:
filter = "English" #@param ["English", "Spanish"]
if languages == "Both":
  for i in idiomas:
    df = pd.DataFrame(np.array(googleplay(app, count, rating, choice, i)),columns=['review'])
    df2 = df.join(pd.DataFrame(df.pop('review').tolist()))
    df3 = pd.concat([df2, df3])
    # Apply the detect_language function to each review
  df3['language'] = df3['content'].apply(detect_language)
  df3.sort_values(by='at', ascending = False, inplace = True)
  if filter == "English":
    df3.sort_values(by='language', ascending = True, inplace = True)
  else:
    df3.sort_values(by='language', ascending = False, inplace = True)

#@markdown * ### Display data?
visualize = "Yes" #@param ["Yes", "No"]
if visualize == "Yes":
  display(df2)
else:
  print("No display")

#@markdown * ### Save to csv?
csv = "Yes" #@param ["Yes", "No"]
if csv == "Yes":
  df3.to_csv('googleplay-review.csv') 
else:
  print("No export")


,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,ad250bbc-7f87-4b2c-828a-07f5405ecf9d,uwu,https://play-lh.googleusercontent.com/a/AGNmyx...,el temporizador para recargar vidas termina pe...,5,0,4.44.0,2023-05-02 21:08:45,None,NaT,4.44.0
1,d849d7c1-47e0-43c5-96b0-79b7071b64b4,michael diez,https://play-lh.googleusercontent.com/a-/ACB-R...,es genial👌,5,0,4.43.2,2023-05-02 15:56:53,None,NaT,4.43.2
2,916d3115-0043-4867-9a7e-469e97d7579b,Frank training,https://play-lh.googleusercontent.com/a-/ACB-R...,muy buena aplicación,5,0,4.43.2,2023-05-02 12:44:54,None,NaT,4.43.2
3,2a269e3a-5244-49e2-bddf-4d825cfd6394,Giovanni Hernandez,https://play-lh.googleusercontent.com/a-/ACB-R...,Es una app fácil de entender muy recomendable ...,5,0,4.43.2,2023-05-02 03:00:31,None,NaT,4.43.2
4,77cbcf66-55be-47fe-a54b-f233303b11ca,SERGIO MUÑOZ,https://play-lh.googleusercontent.com/a-/ACB-R...,Excelente aplicación,5,0,4.44.0,2023-05-01 12:03:42,None,NaT,4.44.0
...,...,...,...,...,...,...,...,...,...,...,...
95,4450d840-ab33-47a4-832c-0b9899c86ee7,Angeline Valdiviezo,https://play-lh.googleusercontent.com/a-/ACB-R...,Buena elección para aprender y ser parte de la...,5,0,4.42.0,2023-04-17 03:27:32,None,NaT,4.42.0
96,5f1c5a0a-7be5-4f36-8c66-b4d04d932a80,Ivan Monzon,https://play-lh.googleusercontent.com/a-/ACB-R...,Una manera muy didáctica y fácil de comenzar a...,5,0,4.38.3,2023-04-17 00:09:24,None,NaT,4.38.3
97,6f3756f8-93f6-4f7e-ac36-9a4bcd99372c,Emanuel Arévalo Martínez,https://play-lh.googleusercontent.com/a-/ACB-R...,Podra ser todo lo buena que sea pero no tengo ...,1,0,4.41.1,2023-04-16 21:47:10,"Hey, Thanks for your feedback. We work tireles...",2023-04-17 11:52:38,4.41.1
98,cc18fd2b-6ace-4892-90a9-b27dcc10ba92,Héctor Santiago Sandoval Jiménez,https://play-lh.googleusercontent.com/a-/ACB-R...,es una aplicación muy util,5,0,4.35.1,2023-04-16 20:58:26,None,NaT,4.35.1
